In [1]:
import sys
sys.path.append("src")
import os

In [2]:
import libfeatureselection
import libpybiofeature
from libmodel import t3mm

2023-07-23 11:57:37.075422: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-23 11:57:37.077695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-23 11:57:37.077702: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-23 11:57:37.617111: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-07-23 11:57:37.617124: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to c

In [3]:
import numpy as np
import pandas as pd
import json
from Bio import SeqIO

In [4]:
import utils
work_Dir = utils.workdir.workdir(os.getcwd(), 4)
work_Dir

'/mnt/md0/Users/georgezhao/Source/TxSEml_Addon'

In [5]:
protype, cter_bool, db_size = 2, False, 'small'
Tx_arg = {
    "type": f'T{protype}',
    # 'seq_id': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id.json']),
    'shufflesplit_index_file': os.path.join(work_Dir, *['data', 'db', f'T{protype}', 'seq_id_shufflesplit.json']),
    'fasta': {
        'cter': cter_bool,
        'sp': {
            'p': "data/T2SE/spT2SE.fasta",
            'n': "data/T2SE/sp_paired_non_t2se.fasta"
        },
        'nosp': {
            'p': "data/T2SE/nospT2SE.fasta",
            'n': "data/T2SE/nosp_paired_non_t2se.fasta"
        },
    },
    'possum': {
        'index': "out/libfeatureselection/SP_feature_research/featuredb/possum/possum_index.json",
        'pssm_fdb_pattern': "out/libfeatureselection/SP_feature_research/featuredb/possum/{zipid}_pssm_features.zip",
        'pssm_rdb_pattern': "out/libfeatureselection/SP_feature_research/featuredb/possum/{zipid}_pssm_files.zip"
    },
    'model': {
        'size': db_size,
        'cter': cter_bool,
        "path_to_save_dir": f"out/libfeatureselection/T2/model/Bastion6_SP/model/"
    }
}

In [6]:
seq_id_dict = {
    "sp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['sp']['n'], "fasta") ],
    },
    "nosp": {
        "p": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['p'], "fasta") ],
        "n": [ seq.id for seq in SeqIO.parse(Tx_arg['fasta']['nosp']['n'], "fasta") ],
    },
}
with open(f"data/T2SE/seq_id.json", "w+", encoding="UTF-8") as f:
    json.dump(
        obj=seq_id_dict,
        fp=f,
    )

In [7]:
feature_data_set = []

In [8]:
feature_data_set.append({
    "name": "AAC",
    "sp_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_acc_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_AAC: 100%|██████████| 11/11 [00:00<00:00, 58327.87it/s]


In [9]:
feature_data_set.append({
    "name": "DAC",
    "sp_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_dac_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_DAC: 100%|██████████| 11/11 [00:00<00:00, 9491.33it/s]


In [10]:
feature_data_set.append({
    "name": "QSO",
    "sp_p": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p',
        cter=Tx_arg['model']['cter']
    ),
    "sp_n": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n',
        cter=Tx_arg['model']['cter']
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p',
        cter=Tx_arg['model']['cter']
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_qso_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n',
        cter=Tx_arg['model']['cter']
    )
})

nosp_n_QSO: 100%|██████████| 11/11 [00:00<00:00, 159.81it/s]


In [11]:
feature_data_set.append({
    "name": "CTDC",
    "sp_p": libpybiofeature.featurebuilder.build_CTDC_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_CTDC_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_CTDC_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_CTDC_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_CTDC: 100%|██████████| 11/11 [00:00<00:00, 18469.71it/s]


In [12]:
feature_data_set.append({
    "name": "CTDT",
    "sp_p": libpybiofeature.featurebuilder.build_CTDT_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p'
    ),
    "sp_n": libpybiofeature.featurebuilder.build_CTDT_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n'
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_CTDT_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p'
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_CTDT_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n'
    )
})

nosp_n_CTDT: 100%|██████████| 11/11 [00:00<00:00, 955.09it/s]


In [13]:
feature_data_set.append({
    "name": "B62",
    "sp_p": libpybiofeature.featurebuilder.build_b62_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        seq_id_list=seq_id_dict['sp']['p'],
        desc='sp_p',
        cter=Tx_arg['fasta']['cter']
    ),
    "sp_n": libpybiofeature.featurebuilder.build_b62_feature(
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        seq_id_list=seq_id_dict['sp']['n'],
        desc='sp_n',
        cter=Tx_arg['fasta']['cter']
    ),
    "nosp_p": libpybiofeature.featurebuilder.build_b62_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        seq_id_list=seq_id_dict['nosp']['p'],
        desc='nosp_p',
        cter=Tx_arg['fasta']['cter']
    ),
    "nosp_n": libpybiofeature.featurebuilder.build_b62_feature(
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        seq_id_list=seq_id_dict['nosp']['n'],
        desc='nosp_n',
        cter=Tx_arg['fasta']['cter']
    )
})

nosp_n_B62: 100%|██████████| 11/11 [00:00<00:00, 16430.68it/s]


In [14]:
possum_index_dict = None
with open(Tx_arg['possum']['index'], 'r', encoding='UTF-8') as f:
    possum_index_dict = json.load(f)

feature_data_set.append({
    "name": "PSSM_feature",
    "sp_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['sp_p'],
        feature_name_list=['dpc_pssm', 's_fpssm', 'pse_pssm', ],
        path_to_fasta=Tx_arg['fasta']['sp']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['sp']['p'], :],
    "sp_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['sp_n'],
        feature_name_list=['dpc_pssm', 's_fpssm', 'pse_pssm', ],
        path_to_fasta=Tx_arg['fasta']['sp']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['sp']['n'], :],
    "nosp_p": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['nosp_p'],
        feature_name_list=['dpc_pssm', 's_fpssm', 'pse_pssm', ],
        path_to_fasta=Tx_arg['fasta']['nosp']['p'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['nosp']['p'], :],
    "nosp_n": libpybiofeature.libdataloader.pssm_tools.get_all_pssm_feature(
        possum_index_list=possum_index_dict['data']['nosp_n'],
        feature_name_list=['dpc_pssm', 's_fpssm', 'pse_pssm', ],
        path_to_fasta=Tx_arg['fasta']['nosp']['n'],
        path_to_with_pattern=Tx_arg['possum']['pssm_fdb_pattern']
    ).loc[seq_id_dict['nosp']['n'], :],
})

In [15]:
def merge_pn_dataset(
    p_f: pd.DataFrame,
    p_l: np.ndarray,
    n_f: pd.DataFrame,
    n_l: np.ndarray
):
    t_f = pd.concat([p_f, n_f])
    t_l = np.concatenate([p_l, n_l])

    return t_f, t_l

# 合并feature_selected成数据集
data_set_split = {
    datatype: pd.concat([
        item[datatype] for item in feature_data_set
    ], axis=1)
    for datatype in ["sp_p", "sp_n", "nosp_p", "nosp_n"]
}
label_set_split = {
    datatype: np.ones(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_p", "nosp_p",]
} | {
    datatype: np.zeros(
        shape=(feature_data_set[0][datatype].shape[0], ))
    for datatype in ["sp_n", "nosp_n",]
}

t_f, t_l = merge_pn_dataset(
    p_f=data_set_split["sp_p"],
    p_l=label_set_split["sp_p"],
    n_f=data_set_split["sp_n"],
    n_l=label_set_split["sp_n"],
)
v_f, v_l = merge_pn_dataset(
    p_f=data_set_split["nosp_p"],
    p_l=label_set_split["nosp_p"],
    n_f=data_set_split["nosp_n"],
    n_l=label_set_split["nosp_n"],
)

In [16]:
# 丢入model中进行训练
model_path = f"{Tx_arg['model']['path_to_save_dir']}/model/"
os.makedirs(
    model_path,
    exist_ok=True
)
model_path

'out/libfeatureselection/T2/model/Bastion6_SP/model//model/'

In [17]:
from libfeatureselection import model, model_space
n_jobs = (
    (os.cpu_count() - 2)
    if "n_jobs" not in os.environ or os.environ['n_jobs'] == "" else
    int(os.environ['n_jobs'])
)
n_jobs

30

In [29]:
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
import math
import pickle
import gzip
import itertools

from src.libmodel import common, model_optimite

import matplotlib as mpl

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'
mpl.rcParams['pdf.use14corefonts'] = False
# mpl.rcParams['pdf.usecorefonts'] = True
mpl.rcParams['pdf.compression'] = 9

import matplotlib.pyplot as plt
import scienceplots

plt.style.use(['science', 'nature'])


from sklearn.metrics import roc_curve, confusion_matrix, precision_score, accuracy_score, f1_score, matthews_corrcoef, auc

def get_evaluation(label: list, pred: list, pro_cutoff: float = None):
    pred = np.nan_to_num(
        pred, copy=True, nan=0.0
    )
    fpr, tpr, thresholds = roc_curve(label, pred)
    if pro_cutoff is None:
        best_one_optimal_idx = np.argmax(tpr - fpr)
        pro_cutoff = thresholds[best_one_optimal_idx]
    pred_l = [1 if i >= pro_cutoff else 0 for i in pred]
    confusion_matrix_1d = confusion_matrix(label, pred_l).ravel()
    confusion_dict = {N: n for N, n in zip(['tn', 'fp', 'fn', 'tp'], list(
        confusion_matrix_1d * 2 / np.sum(confusion_matrix_1d)))}
    evaluation = {
        "accuracy": accuracy_score(label, pred_l),
        "precision": precision_score(label, pred_l),
        "f1_score": f1_score(label, pred_l),
        "mmc": matthews_corrcoef(label, pred_l),
        "rocAUC": auc(fpr, tpr),
        "specificity": confusion_dict['tn'] / (confusion_dict['tn'] + confusion_dict['fp']),
        "sensitivity": confusion_dict['tp'] / (confusion_dict['tp'] + confusion_dict['fn']),
        # "confusion_matrix": confusion_dict,
        # "_roc_Data": {'fpr': list(fpr), 'tpr': list(tpr)},
        'pro_cutoff': pro_cutoff
    }
    return evaluation


def plot_roc_curve(target, pred, path_to_: str):
    fpr, tpr, thresholds = roc_curve(target, pred)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(19.2 / 4, 10.8 / 4))
    plt.axis('square')
    plt.plot(
        fpr, tpr, color='red', lw=2,
        label='ROC curve (area = %0.2f)' % roc_auc
    )
    plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic (ROC) curve')
    plt.legend(loc="lower right")

    plt.savefig(f"{path_to_}", transparent=True)
    plt.clf()

submodel_choise = None

svm_param_o = {
    "gamma": [2**i for i in range(-6, 6 + 1)],
    "C": [2**i for i in range(-6, 6 + 1)],
}

class svm_optim_chiosed(model_optimite.svm_optim):
    svm_choised_scale = 0.75

    def __init__(self, param_o, cv, default_param={
        'verbose': False,
        'kernel': 'rbf',
        'probability': True
    }) -> None:
        super().__init__(param_o, cv, default_param=default_param)
        self.col_name = None
        self.origin_col = None
        pass

    def _chiose_col_name(self, X: pd.DataFrame, scale: float = svm_choised_scale, force: bool = False):
        self.origin_col = X.columns
        self.col_name = np.random.choice(
            X.columns, size=max(1, math.floor(len(X.columns) * scale)), replace=False)
        return self

    def _get_col_named(self, X: pd.DataFrame):
        if self.origin_col.tolist() != X.columns.tolist():
            print(self.origin_col, X.columns)
            raise RuntimeError('self.origin_col != X.columns')
        return X.loc[:, self.col_name]

    def best_fit(self, X, y, verbose, n_jobs):
        self._chiose_col_name(X)
        return super().best_fit(
            X=self._get_col_named(X),
            y=y,
            verbose=verbose,
            n_jobs=n_jobs
        )

    def fit(self, X, y):
        # self._chiose_col_name(X)
        return super().fit(X, y)

    def predict_proba(self, X):
        return super().predict_proba(
            X=self._get_col_named(X)
        )

    def find_parm(self, X, y, n_jobs, verbose):
        return super().find_parm(X, y, n_jobs, verbose=verbose)


class Bastion6_Model(common.Model_Final):

    def __init__(self, N, cv, desc):
        super().__init__(cv, desc=desc)
        self.num_of_clsif = N
        self.model_svm_2d_list = None

        self.feature_dividend_list = None
        pass

    def tranmodel(self, f, l, feature_dividend_list=[0, 520, 1198, None]):
        super().tranmodel(f, l)

        self.feature_dividend_list = feature_dividend_list

        numofcls_for_group = math.floor(
            self.num_of_clsif / (
                len(self.feature_dividend_list) - 1
            )
        )

        self.model_svm_2d_list = list()
        for feature_group_index in range(
            1,
            len(self.feature_dividend_list)
        ):
            model_svm_1d_list = list()
            for _ in range(numofcls_for_group):
                model_svm_1d_list.append(
                    svm_optim_chiosed(param_o=svm_param_o, cv=self.cv).best_fit(
                        f.iloc[
                            :,
                            self.feature_dividend_list[
                                feature_group_index - 1
                            ]:
                            self.feature_dividend_list[
                                feature_group_index
                            ]
                        ],
                        l,
                        verbose=-1,
                        n_jobs=n_jobs
                    )
                )
            self.model_svm_2d_list.append(model_svm_1d_list)

        return self

    def predict(self, f):
        super().predict(f)
        numofcls_for_group = math.floor(
            self.num_of_clsif / (
                len(self.feature_dividend_list) - 1
            )
        )
        result = np.stack(
            list(
                itertools.chain(*[
                    [
                        m.predict_proba(
                            f.iloc[
                                :,
                                self.feature_dividend_list[
                                    feature_group_index - 1
                                ]:
                                self.feature_dividend_list[
                                    feature_group_index
                                ]
                            ]
                        )
                        for m in self.model_svm_2d_list[feature_group_index - 1]
                    ]
                    for feature_group_index in range(
                        1,
                        len(self.feature_dividend_list)
                    )
                ])
            )
        ).T
        if submodel_choise is None:
            return result.sum(axis=1) / self.num_of_clsif
        if isinstance(submodel_choise, int) == False or submodel_choise < 0 or submodel_choise >= (len(self.feature_dividend_list) - 1):
            raise ValueError(f"Wrong submodel_choise: {submodel_choise}")
        return result[:, submodel_choise]


class Bastion6_Trainer:
    def __init__(self, ) -> None:
        self.classifier_name = "Bastion6"
        self.classifier_class = Bastion6_Model
        self.classifier_param_dict = {
            "desc": "Bastion6_SP",
            "cv": 5,
            "N": 100,
        }

        self.model = None
        self.train_best_predicted_pair = None
        self.train_best_5C_predicted_pair = None
        self.best_predicted_pair = None
        self.best_5C_predicted_pair = None
        self.start_to_train_time = datetime.now()
        self.end_of_train_time = None
        pass

    def find_best(
        self,
        X: np.ndarray,
        y: np.ndarray,
        validation: tuple,
        merge_validate: bool
    ):

        self.model = self.classifier_class(
            **self.classifier_param_dict
        )
        self.model.tranmodel(
            pd.DataFrame(X, columns=list(range(2038))),
            y,
        )
        self.best_predicted_pair = [
            np.nan_to_num(self.model.predict(
                pd.DataFrame(validation[0], columns=list(range(2038))),
            ), nan=0.0),
            validation[1]
        ]
        self.train_best_predicted_pair = [
            np.nan_to_num(self.model.predict(
                pd.DataFrame(X, columns=list(range(2038))),
            ), nan=0.0),
            y
        ]

        # 5倍交叉验证
        # 合并数据
        if merge_validate == True:
            full_X = np.concatenate([
                X, validation[0]
            ])
            full_y = np.concatenate([
                y, validation[1]
            ])
        else:
            full_X = np.concatenate([
                X,
            ])
            full_y = np.concatenate([
                y,
            ])

        # 跑模型
        self.best_5C_predicted_pair = []
        self.train_best_5C_predicted_pair = []
        for Kfold_id, (train_id, test_id) in enumerate(
            StratifiedKFold(
                n_splits=5,
                shuffle=True,
                random_state=42
            ).split(full_X, full_y)
        ):

            # 定义模型并加载参数
            fiveC_model = self.classifier_class(
                **self.classifier_param_dict,
            )

            fiveC_model.tranmodel(
                pd.DataFrame(full_X[train_id], columns=list(range(2038))),
                full_y[train_id],
            )

            # 预测并记录
            self.best_5C_predicted_pair.append([
                np.nan_to_num(fiveC_model.predict(
                    pd.DataFrame(full_X[test_id], columns=list(range(2038))),
                ), nan=0.0),
                full_y[test_id]
            ])
            self.train_best_5C_predicted_pair.append([
                np.nan_to_num(fiveC_model.predict(
                    pd.DataFrame(full_X[train_id], columns=list(range(2038))),
                ), nan=0.0),
                full_y[train_id]
            ])

        return self

    def get_summary(self, path_to_dir: str = None):
        os.makedirs(path_to_dir, exist_ok=True)
        model_path = "-"
        if "SAVE_MODEL" in os.environ and os.environ['SAVE_MODEL'] == "1":

            model_path = f"{path_to_dir}/{self.classifier_name}.pkl"
            if path_to_dir is not None:
                with gzip.open(model_path, "wb") as f:
                    pickle.dump(
                        self.grid_search, f
                    )

        model_score_path = f"{path_to_dir}/{self.classifier_name}_score.pkl"
        if path_to_dir is not None:
            with gzip.open(model_score_path, "wb") as f:
                pickle.dump(
                    {
                        "best_predicted_pair": self.best_predicted_pair,
                        "best_5C_predicted_pair": self.best_5C_predicted_pair,
                    }, f
                )
            with gzip.open(model_score_path + ".train", "wb") as f:
                pickle.dump(
                    {
                        "best_predicted_pair": self.train_best_predicted_pair,
                        "best_5C_predicted_pair": self.train_best_5C_predicted_pair,
                    }, f
                )
        else:
            model_score_path = "-"

        plot_roc_curve(
            target=self.best_predicted_pair[1],
            pred=self.best_predicted_pair[0],
            path_to_=f"{path_to_dir}/{self.classifier_name}.pdf"
        )

        model_information = {
            "Classifier_Name": self.classifier_name,
            "Optimitied_Param": dict(),
            "Score": model_score_path,
            "Model_Path": model_path,
            "TimeToStartFit": self.start_to_train_time.strftime("%Y-%m-%d %H:%M:%S")
        }

        training_testing_performance = get_evaluation(
            label=self.best_predicted_pair[1],
            pred=self.best_predicted_pair[0],
        )

        # 计算5C中的平均表现
        FiveFold_result = {}
        for keys in training_testing_performance.keys():
            value_list = []
            for item in self.best_5C_predicted_pair:

                item_performance = get_evaluation(
                    label=item[1],
                    pred=item[0],
                )
                value_list.append(item_performance[keys])

            if keys == "pro_cutoff":
                FiveFold_result[keys] = value_list
            else:
                FiveFold_result[keys] = sum(value_list) / len(value_list)

        self.end_of_train_time = datetime.now()
        model_information["TimeOfSummary"] = self.end_of_train_time.strftime(
            "%Y-%m-%d %H:%M:%S"
        )
        model_information["TimeSpend"] = str(
            self.end_of_train_time - self.start_to_train_time
        )

        return model_information, training_testing_performance, FiveFold_result

In [30]:
Bastion6_Trainer().find_best(
    X=t_f.values,
    y=t_l,
    validation=(
        v_f.values,
        v_l,
    ),
    merge_validate=False
).get_summary(
    path_to_dir="out/libfeatureselection/T2/Bastion6_SP/"
)

({'Classifier_Name': 'Bastion6',
  'Optimitied_Param': {},
  'Score': 'out/libfeatureselection/T2/Bastion6_SP//Bastion6_score.pkl',
  'Model_Path': '-',
  'TimeToStartFit': '2023-07-23 13:05:03',
  'TimeOfSummary': '2023-07-23 13:10:27',
  'TimeSpend': '0:05:23.866533'},
 {'accuracy': 0.7272727272727273,
  'precision': 0.6666666666666666,
  'f1_score': 0.7692307692307692,
  'mmc': 0.48795003647426655,
  'rocAUC': 0.6363636363636364,
  'specificity': 0.5454545454545454,
  'sensitivity': 0.9090909090909091,
  'pro_cutoff': 0.3506602011290099},
 {'accuracy': 0.9431372549019608,
  'precision': 0.9377777777777776,
  'f1_score': 0.9433918128654971,
  'mmc': 0.8894057807415156,
  'rocAUC': 0.9589506172839506,
  'specificity': 0.9305555555555556,
  'sensitivity': 0.9527777777777778,
  'pro_cutoff': [0.6187588966407137,
   0.5590046834255721,
   0.5184827993283286,
   0.5270531227586792,
   0.5222636096359252]})

<Figure size 480x270 with 0 Axes>